In [ ]:
import os
from smolagents import AzureOpenAIServerModel, TransformersModel

MODE = 'local'  # or 'azure'
if MODE == 'azure':
    model = AzureOpenAIServerModel(
        model_id=os.environ.get("AZURE_OPENAI_MODEL", ""),
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
        api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
        api_version=os.environ.get("OPENAI_API_VERSION", "")
    )
if MODE == 'local':
    model = TransformersModel(
        model_id="Qwen/Qwen2.5-Coder-32B-Instruct",
        device="cuda" if os.environ.get("CUDA_VISIBLE_DEVICES", "") else "cpu",
        max_length=512,
        temperature=0.7,
        top_p=0.9,
        top_k=50
    )

In [11]:
from pathlib import Path
from medminer.data import Document

docs = []
for file in (Path.cwd().parent / "data" / "pre_medications").glob("*.txt"):
    with open(file, "r") as f:
        docs.append(Document(file.stem, f.read()))

In [3]:
from medminer.task.medication import medication_task

for doc in docs[:1]:
    medication_task.run(model, doc.content)